In [1]:
import pandas as pd
from pathlib import Path
import re

# --- chemins (à adapter si besoin) ---
CSV1 = Path("combined_journalier_capi.csv")   # ta 1ère table
CSV2 = Path("Cours_BRVM.csv")                 # ta 2ème table
OUT  = Path("Base_complète.csv")

# --- lecture ---
df1 = pd.read_csv(CSV1, encoding="utf-8-sig")
df2 = pd.read_csv(CSV2, encoding="utf-8-sig")

# --- nettoyage Ticker & Date ---
def clean_ticker(s: pd.Series) -> pd.Series:
    # supprime le suffixe " J" éventuel + espaces parasites
    return (
        s.astype(str)
         .str.strip()
         .str.replace(r"\s*J$", "", regex=True)
         .str.upper()
    )

def clean_date(s: pd.Series) -> pd.Series:
    # normalise au format AAAA-MM-JJ
    d = pd.to_datetime(s, errors="coerce", dayfirst=False)
    return d.dt.strftime("%Y-%m-%d")

# 1) sur la table 1 : enlever " J" dans chaque ticker + normaliser Date
if "Ticker" in df1.columns:
    df1["Ticker"] = clean_ticker(df1["Ticker"])
else:
    raise ValueError("La 1ère table doit contenir une colonne 'Ticker'.")

if "Date" in df1.columns:
    df1["Date"] = clean_date(df1["Date"])
else:
    raise ValueError("La 1ère table doit contenir une colonne 'Date'.")

# 2) sur la table 2 : normaliser Ticker + Date aussi
if "Ticker" in df2.columns:
    df2["Ticker"] = clean_ticker(df2["Ticker"])
else:
    raise ValueError("La 2ème table doit contenir une colonne 'Ticker'.")

if "Date" in df2.columns:
    df2["Date"] = clean_date(df2["Date"])
else:
    raise ValueError("La 2ème table doit contenir une colonne 'Date'.")

# 3) retirer les colonnes à exclure côté table 2
cols_a_exclure = {
    "Global capitalization",
    "rang_capi",
    "Number of shares",
    "Global capitalization (%)",
}
colonnes_df2 = [c for c in df2.columns if c not in cols_a_exclure]

# on évite les doublons de colonnes Ticker/Date au merge
colonnes_df2_sans_keys = [c for c in colonnes_df2 if c not in ("Ticker", "Date")]

# 4) merge sur (Ticker, Date)
# par défaut je fais un left join sur la table 1 (ta base principale)
df_merged = df1.merge(
    df2[["Ticker", "Date"] + colonnes_df2_sans_keys],
    on=["Ticker", "Date"],
    how="left",
    validate="m:1"  # chaque (Ticker, Date) de df1 doit correspondre à 0/1 ligne de df2
)

# 5) optionnel : trier les colonnes (facultatif)
# on garde l’ordre de df1 puis on ajoute ce qui vient de df2
cols_final = list(df1.columns) + [c for c in colonnes_df2_sans_keys if c not in df1.columns]
df_merged = df_merged.reindex(columns=cols_final)

# 6) export
df_merged.to_csv(OUT, index=False, encoding="utf-8-sig")

print(f"✅ Fusion terminée. Fichier exporté : {OUT.resolve()}")


✅ Fusion terminée. Fichier exporté : /Users/jeanjoelgoli/Documents/FINANCE/Travaux BRVM/Base_complète.csv


In [2]:
import pandas as pd
from pathlib import Path

# --- chemins d'entrée/sortie ---
CSV1 = Path("combined_journalier_capi.csv")
CSV2 = Path("Cours_BRVM.csv")
CSV3 = Path("richbourse_societes.csv")
CSV4 = Path("dividende_histo.csv")
OUT  = Path("Base_complète.csv")

# --- helpers ---
def clean_ticker_col(series: pd.Series) -> pd.Series:
    return (
        series.astype(str)
              .str.strip()
              .str.replace(r"\s*J$", "", regex=True)
              .str.upper()
    )

def clean_date_col(series: pd.Series) -> pd.Series:
    # Si tes CSV sont au format français, mets dayfirst=True
    d = pd.to_datetime(series, errors="coerce", dayfirst=False)
    return d.dt.strftime("%Y-%m-%d")

# --- lecture ---
df1 = pd.read_csv(CSV1, encoding="utf-8-sig")
df2 = pd.read_csv(CSV2, encoding="utf-8-sig")
df3 = pd.read_csv(CSV3, encoding="utf-8-sig")
df4 = pd.read_csv(CSV4, encoding="utf-8-sig")

# --- nettoyage table 1 (gauche) ---
df1["Ticker"] = clean_ticker_col(df1["Ticker"])
df1["Date"]   = clean_date_col(df1["Date"])

# --- nettoyage table 2 ---
df2["Ticker"] = clean_ticker_col(df2["Ticker"])
df2["Date"]   = clean_date_col(df2["Date"])

cols_exclure = {
    "Global capitalization",
    "rang_capi",
    "Number of shares",
    "Global capitalization (%)",
}
cols_df2_keep = [c for c in df2.columns if c not in cols_exclure]
cols_df2_payload = [c for c in cols_df2_keep if c not in ("Ticker", "Date")]

# rendre unique sur (Ticker, Date)
df2_right = df2[["Ticker", "Date"] + cols_df2_payload].sort_values(["Ticker", "Date"])
df2_right = df2_right.drop_duplicates(["Ticker", "Date"], keep="last")

df_merged = df1.merge(
    df2_right,
    on=["Ticker", "Date"],
    how="left",
    validate="m:1"
)

# --- nettoyage table 3 ---
df3 = df3.rename(columns={c: "Ticker" for c in df3.columns if c.lower() == "ticker"})
df3["Ticker"] = clean_ticker_col(df3["Ticker"])
cols_df3_payload = [c for c in df3.columns if c != "Ticker"]

# rendre unique sur (Ticker)
df3_right = df3[["Ticker"] + cols_df3_payload].sort_values(["Ticker"])
df3_right = df3_right.drop_duplicates(["Ticker"], keep="last")

df_merged = df_merged.merge(
    df3_right,
    on="Ticker",
    how="left",
    validate="m:1"
)

# --- nettoyage table 4 (dividendes) ---
df4 = df4.rename(columns={c: "Ticker" for c in df4.columns if c.lower() == "ticker"})
df4["Ticker"] = clean_ticker_col(df4["Ticker"])
df4["Ex-dividende"]  = clean_date_col(df4["Ex-dividende"])
df4["Date paiement"] = clean_date_col(df4["Date paiement"])

cols_df4_payload = [c for c in df4.columns if c not in ("Ticker", "Ex-dividende")]

# rendre unique sur (Ticker, Ex-dividende) puis renommer en Date
df4_tmp = df4.rename(columns={"Ex-dividende": "Date"})
df4_right = df4_tmp[["Ticker", "Date"] + cols_df4_payload].sort_values(["Ticker", "Date"])
df4_right = df4_right.drop_duplicates(["Ticker", "Date"], keep="last")

df_merged = df_merged.merge(
    df4_right,
    on=["Ticker", "Date"],
    how="left",
    validate="m:1"
)

# --- vérification NA ---
na_mask = df_merged.isna().any(axis=1)
nb_lignes_na = na_mask.sum()
if nb_lignes_na == 0:
    print("✅ Aucune valeur manquante (NA) dans la table finale.")
else:
    print(f"⚠️ {nb_lignes_na} lignes contiennent au moins un NA.")
    tickers_na = df_merged.loc[na_mask, "Ticker"].dropna().unique()
    print("Tickers avec NA :", tickers_na)

# --- export ---
df_merged.to_csv(OUT, index=False, encoding="utf-8-sig")
print(f"\n✅ Export terminé : {OUT.resolve()}")


/var/folders/0n/svq7n_mn2lg1fv5jbk66hzg00000gn/T/ipykernel_2667/1506569166.py:22: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  d = pd.to_datetime(series, errors="coerce", dayfirst=False)
/var/folders/0n/svq7n_mn2lg1fv5jbk66hzg00000gn/T/ipykernel_2667/1506569166.py:22: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  d = pd.to_datetime(series, errors="coerce", dayfirst=False)


⚠️ 147859 lignes contiennent au moins un NA.
Tickers avec NA : ['ABJC' 'BICB' 'BICC' 'BNBC' 'BOAB' 'BOABF' 'BOAC' 'BOAM' 'BOAN' 'BOAS'
 'CABC' 'CBIBF' 'CFAC' 'CIEC' 'ECOC' 'ETIT' 'FTSC' 'LNBB' 'NEIC' 'NSBC'
 'NTLC' 'ONTBF' 'ORAC' 'ORGT' 'PALC' 'PRSC' 'SAFC' 'SCRC' 'SDCC' 'SDSC'
 'SEMC' 'SGBC' 'SHEC' 'SIBC' 'SICC' 'SIVC' 'SLBC' 'SMBC' 'SNTS' 'SOGC'
 'SPHC' 'STAC' 'STBC' 'SVOC' 'TTLC' 'TTLS' 'UNLC' 'UNXC']

✅ Export terminé : /Users/jeanjoelgoli/Documents/FINANCE/Travaux BRVM/Base_complète.csv


In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd

SRC = "Base_complète.csv"
DST = "Base_complète_div.csv"

# Charge le fichier
df = pd.read_csv(SRC)

# Assure-toi que les colonnes attendues existent
needed = ["Ticker","Date","Cours Ajuste","Dividende ajusté"]
missing = [c for c in needed if c not in df.columns]
if missing:
    raise ValueError(f"Colonnes manquantes: {missing}")

# Convertit en float
df["Cours Ajuste"] = pd.to_numeric(df["Cours Ajuste"], errors="coerce")
df["Dividende ajusté"] = pd.to_numeric(df["Dividende ajusté"], errors="coerce").fillna(0.0)

# --- 1) Cours Ajuste AddDiv ---
df["Cours Ajuste AddDiv"] = df["Cours Ajuste"] + df["Dividende ajusté"]

# --- 2) Cours Ajuste BackcorrectDiv ---
# Idée : pour chaque Ticker, on calcule un facteur cumulatif rétroactif
dfs = []
for t, dft in df.groupby("Ticker"):
    dft = dft.sort_values("Date")  # ordonne chronologiquement
    dft["PrevCours"] = dft["Cours Ajuste"].shift(1)

    # ratio = (P_{t-1} - Div) / P_{t-1} aux dates ex-div
    dft["ratio"] = 1.0
    mask = dft["Dividende ajusté"] > 0
    dft.loc[mask, "ratio"] = (dft.loc[mask, "PrevCours"] - dft.loc[mask, "Dividende ajusté"]) / dft.loc[mask, "PrevCours"]

    # cumprod rétroactif (du présent vers le passé)
    adj_factor = dft["ratio"][::-1].cumprod()[::-1]
    dft["Cours Ajuste BackcorrectDiv"] = dft["Cours Ajuste"] * adj_factor

    dfs.append(dft.drop(columns=["PrevCours","ratio"]))

df_out = pd.concat(dfs, ignore_index=True)

# Sauvegarde
df_out.to_csv(DST, index=False, encoding="utf-8")
print(f"✔ Fichier exporté : {DST} ({len(df_out)} lignes)")


/var/folders/0n/svq7n_mn2lg1fv5jbk66hzg00000gn/T/ipykernel_2667/3755240380.py:10: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(SRC)


✔ Fichier exporté : Base_complète_div.csv (147859 lignes)
